# Quantum Chem. VQE-MUB Code, V2

Refactored to work with qiskit-nature 0.7.2 and qiskit 1.2.0.
Because everything else breaks all the way to hell.

Written by Ittay Alfassi, based on work from Dekel Meirom

### Required Versions:
- qiskit-nature==0.7.2  (via pip)
- pyscf==2.6.2          (via conda)
- matplotlib==3.9.2     (via conda)
- qiskit==1.2.0         (via pip, automatic when installing qiskit-nature)
- numpy==1.26.4         (built-in with conda)

### Required Infrastructure

- Linux or MacOS. **We highly recommend** to use the supplied dockerfile to set up a debian container.
- Miniconda3 (current version is conda 24.7.1)

## Imports

### General Imports

In [1]:
import qiskit
from qiskit import QuantumCircuit
import numpy as np
from qiskit.circuit import Parameter
from qiskit.quantum_info import Statevector, SparsePauliOp
from qiskit.result import Counts, Result
from typing import Optional, List, Dict, Tuple, Union
from matplotlib import pyplot as plt
import itertools

### Quantum Chemistry Imports

In [2]:
from pyscf import scf
from pyscf import gto

In [3]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

from qiskit_nature.second_q.problems.electronic_structure_problem import ElectronicStructureProblem
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer

from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

## Qubit operator generation functiosn

In [72]:
def get_qubit_op(molecule_str: str, charge: int = 0, basis: str = "sto3g", mapping_name: str = "parity", transformer: Optional[ActiveSpaceTransformer] = None) -> tuple[ElectronicStructureProblem, SparsePauliOp]:
    driver = PySCFDriver(
        atom=molecule_str,
        unit=DistanceUnit.ANGSTROM,
        charge=charge,
        basis=basis,
    )
    problem = driver.run()
    if transformer is not None:
        problem = transformer.transform(problem)
    second_q_op = problem.hamiltonian.second_q_op()
    if mapping_name == "parity":
        mapper = ParityMapper(num_particles=problem.num_particles)
    elif mapping_name == "jordan":
        mapper = JordanWignerMapper(num_particles=problem.num_particles)
    elif mapping_name == "bravi":
        mapper = BravyiKitaevMapper(num_particles=problem.num_particles)
    else:
        print("mapper should be one of parity, jordan or bravi")
        return None
    qubit_op = mapper.map(second_q_op)
    return qubit_op, problem

In [73]:
def get_qubit_op_H2(dist: str, mapping_name: str = "parity") -> tuple[ElectronicStructureProblem, SparsePauliOp]:
    atom_str = f"H .0 .0 .0; H .0 .0 {dist}"
    charge = 0
    basis = 'sto6g'
    return get_qubit_op(atom_str, charge, basis, mapping_name)

def get_qubit_op_heh(dist, mapping_name: str = "parity"):
    atom_str = f"He .0 .0 .0; H .0 .0 {dist}"
    charge = 1
    basis = 'sto3g'
    return get_qubit_op(atom_str, charge, basis, mapping_name)

def get_qubit_op_lih(dist, mapping_name: str = "parity"):
    atom_str = f"Li .0 .0 .0; H .0 .0 {dist}"
    charge = 0
    basis = 'sto3g'
    transformer = ActiveSpaceTransformer(num_electrons=2, num_spatial_orbitals=3, active_orbitals=[1,2,5])
    return get_qubit_op(atom_str, charge, basis, mapping_name, transformer)

In [74]:
def get_exact_energies(elec_struct_prob):
    numpy_solver = NumPyMinimumEigensolver()
    mapper = ParityMapper(num_particles=elec_struct_prob.num_particles)
    calc = GroundStateEigensolver(mapper, numpy_solver)
    temp_res = calc.solve(elec_struct_prob)
    return temp_res, temp_res.total_energies[0], temp_res.nuclear_repulsion_energy

In [75]:
def get_hf_energy_H2(dist):
    temp_mol = gto.M(atom = 'H 0 0 0; H 0 0 ' + str(dist), basis = 'sto6g')
    temp_hf = scf.HF(temp_mol)
    return temp_hf.kernel()

## Generation of Hamiltonians for our experiments

In [76]:
H2_qub, H2_es = get_qubit_op_H2(".75")
HeH_qub, HeH_es = get_qubit_op_heh("1")
LiH_qub, LiH_es = get_qubit_op_lih("1.5")

### Fermionic Hamiltonians

In [77]:
print(H2_es.hamiltonian.second_q_op())
print(HeH_es.hamiltonian.second_q_op())
print(LiH_es.hamiltonian.second_q_op())

Fermionic Operator
number spin orbitals=4, number terms=36
  0.336393232206363 * ( +_0 +_0 -_0 -_0 )
+ 0.331321473942246 * ( +_0 +_1 -_1 -_0 )
+ 0.336393232206363 * ( +_0 +_2 -_2 -_0 )
+ 0.331321473942246 * ( +_0 +_3 -_3 -_0 )
+ 0.09103012464499204 * ( +_0 +_0 -_1 -_1 )
+ 0.09103012464499204 * ( +_0 +_1 -_0 -_1 )
+ 0.09103012464499204 * ( +_0 +_2 -_3 -_1 )
+ 0.09103012464499204 * ( +_0 +_3 -_2 -_1 )
+ 0.09103012464499204 * ( +_1 +_0 -_1 -_0 )
+ 0.09103012464499204 * ( +_1 +_1 -_0 -_0 )
+ 0.09103012464499204 * ( +_1 +_2 -_3 -_0 )
+ 0.09103012464499204 * ( +_1 +_3 -_2 -_0 )
+ 0.331321473942246 * ( +_1 +_0 -_0 -_1 )
+ 0.34867519563338106 * ( +_1 +_1 -_1 -_1 )
+ 0.331321473942246 * ( +_1 +_2 -_2 -_1 )
+ 0.34867519563338106 * ( +_1 +_3 -_3 -_1 )
+ 0.336393232206363 * ( +_2 +_0 -_0 -_2 )
+ 0.331321473942246 * ( +_2 +_1 -_1 -_2 )
+ 0.336393232206363 * ( +_2 +_2 -_2 -_2 )
+ 0.331321473942246 * ( +_2 +_3 -_3 -_2 )
+ 0.09103012464499204 * ( +_2 +_0 -_1 -_3 )
+ 0.09103012464499204 * ( +_2 +_1 -_0

### Qubit Hamiltonians

In [78]:
print(H2_qub)
print(HeH_qub)
print(LiH_qub)

SparsePauliOp(['II', 'IZ', 'ZI', 'ZZ', 'XX'],
              coeffs=[-1.06324002+0.j,  0.38913654+0.j, -0.38913654+0.j, -0.01121274+0.j,
  0.18206025+0.j])
SparsePauliOp(['II', 'IZ', 'ZI', 'ZZ', 'XI', 'XZ', 'IX', 'ZX', 'XX'],
              coeffs=[-3.04506092+0.j,  0.50258052+0.j, -0.50258052+0.j, -0.13894646+0.j,
  0.11926278+0.j,  0.11926145+0.j,  0.11926278+0.j, -0.11926145+0.j,
  0.11714671+0.j])
SparsePauliOp(['IIII', 'IIIZ', 'IIZZ', 'IIZI', 'IIXZ', 'IIXI', 'IZII', 'IZIZ', 'ZXII', 'ZXIZ', 'IXII', 'IXIZ', 'XXII', 'XXIZ', 'YYII', 'YYIZ', 'ZZII', 'ZZIZ', 'XZII', 'XZIZ', 'XIII', 'XIIZ', 'ZIII', 'ZIIZ', 'IIXX', 'IIYY', 'IIZX', 'IIIX', 'IZZX', 'IZIX', 'ZXZX', 'IXZX', 'ZXIX', 'IXIX', 'XXZX', 'YYZX', 'XXIX', 'YYIX', 'ZZZX', 'ZZIX', 'XZZX', 'XIZX', 'XZIX', 'XIIX', 'ZIZX', 'ZIIX', 'IZXX', 'IZYY', 'ZXXX', 'IXXX', 'ZXYY', 'IXYY', 'XXXX', 'YYXX', 'XXYY', 'YYYY', 'ZZXX', 'ZZYY', 'XZXX', 'XIXX', 'XZYY', 'XIYY', 'ZIXX', 'ZIYY', 'IZZZ', 'ZXZZ', 'IXZZ', 'XXZZ', 'YYZZ', 'ZZZZ', 'XZZZ', 'XIZZ', 'ZIZZ'

## Verifying Dekel's code results

In [79]:
H2_ham_0_75 = SparsePauliOp(data= ['II', 'IZ', 'ZI', 'ZZ', 'XX'],
coeffs= [-1.06324002+0.j,  0.38913654+0.j, -0.38913654+0.j, -0.01121274+0.j,  0.18206025+0.j])

HeH_ham_1 = SparsePauliOp(data= ['II', 'IZ', 'IX', 'ZI', 'XI', 'ZZ', 'ZX', 'XZ', 'XX'],
coeffs= [-3.04506092+0.j,  0.50258052+0.j,  0.11926278+0.j, -0.50258052+0.j, 0.11926278+0.j, -0.13894646+0.j, -0.11926145+0.j,  0.11926145+0.j,  0.11714671+0.j])

LiH_ham_1_5 = SparsePauliOp(
    data= ['IIII', 'IIIZ', 'IIZX', 'IIIX', 'IIXX', 'IIYY', 'IIZZ', 'IIXZ', 'IIXI', 'IIZI', 'IZII', 'ZXII', 'IXII', 'XXII', 'YYII',
    'ZZII', 'XZII', 'XIII', 'ZIII', 'IZIZ', 'IZZX', 'IZIX', 'IZXX', 'IZYY', 'ZXIZ', 'IXIZ', 'ZXZX', 'IXZX', 'ZXIX', 'IXIX', 'ZXXX',
    'IXXX', 'ZXYY', 'IXYY', 'XXIZ', 'YYIZ', 'XXZX', 'YYZX', 'XXIX', 'YYIX', 'XXXX', 'YYXX', 'XXYY', 'YYYY', 'ZZIZ', 'ZZZX', 'ZZIX',
    'ZZXX', 'ZZYY', 'XZIZ', 'XIIZ', 'XZZX', 'XIZX', 'XZIX', 'XIIX', 'XZXX', 'XIXX', 'XZYY', 'XIYY', 'ZIIZ', 'ZIZX', 'ZIIX', 'ZIXX',
    'ZIYY', 'IZZZ', 'IZXZ', 'IZXI', 'ZXZZ', 'IXZZ', 'ZXXZ', 'IXXZ', 'ZXXI', 'IXXI', 'XXZZ', 'YYZZ', 'XXXZ', 'YYXZ', 'XXXI', 'YYXI',
    'ZZZZ', 'ZZXZ', 'ZZXI', 'XZZZ', 'XIZZ', 'XZXZ', 'XIXZ', 'XZXI', 'XIXI', 'ZIZZ', 'ZIXZ', 'ZIXI', 'IZZI', 'ZXZI', 'IXZI', 'XXZI',
    'YYZI', 'ZZZI', 'XZZI', 'XIZI', 'ZIZI'], 
coeffs = [-1.99754128e-01+0.j, -9.17966069e-02+0.j, -2.73410751e-03+0.j,
        2.73410751e-03+0.j, -3.09895035e-04+0.j,  3.09895035e-04+0.j,  -2.11959340e-01+0.j,  1.95776538e-02+0.j,  1.95776538e-02+0.j,
        3.71356404e-01+0.j,  9.17966069e-02+0.j,  2.73410751e-03+0.j, 2.73410751e-03+0.j, -3.09895035e-04+0.j,  3.09895035e-04+0.j,
       -2.11959340e-01+0.j, -1.95776538e-02+0.j,  1.95776538e-02+0.j, -3.71356404e-01+0.j, -1.23570872e-01+0.j,  1.17336239e-02+0.j,
       -1.17336239e-02+0.j,  3.30587286e-02+0.j, -3.30587286e-02+0.j, 1.17336239e-02+0.j,  1.17336239e-02+0.j, -3.03465683e-03+0.j,
       -3.03465683e-03+0.j,  3.03465683e-03+0.j,  3.03465683e-03+0.j, -8.37336142e-03+0.j, -8.37336142e-03+0.j,  8.37336142e-03+0.j,
        8.37336142e-03+0.j, -3.30587286e-02+0.j,  3.30587286e-02+0.j, 8.37336142e-03+0.j, -8.37336142e-03+0.j, -8.37336142e-03+0.j,
        8.37336142e-03+0.j,  3.07383272e-02+0.j, -3.07383272e-02+0.j, -3.07383272e-02+0.j,  3.07383272e-02+0.j,  5.66560676e-02+0.j,
        1.54067009e-03+0.j, -1.54067009e-03+0.j,  2.36793690e-03+0.j, -2.36793690e-03+0.j, -1.27339140e-02+0.j,  1.27339140e-02+0.j,
        2.11113767e-03+0.j, -2.11113767e-03+0.j, -2.11113767e-03+0.j, 2.11113767e-03+0.j,  7.76444118e-03+0.j, -7.76444118e-03+0.j,
       -7.76444118e-03+0.j,  7.76444118e-03+0.j,  1.14339547e-01+0.j, -1.05401874e-02+0.j,  1.05401874e-02+0.j, -3.51167704e-02+0.j,
        3.51167704e-02+0.j, -5.66560676e-02+0.j, -1.27339140e-02+0.j, -1.27339140e-02+0.j, -1.54067009e-03+0.j, -1.54067009e-03+0.j,
        2.11113767e-03+0.j,  2.11113767e-03+0.j,  2.11113767e-03+0.j, 2.11113767e-03+0.j,  2.36793690e-03+0.j, -2.36793690e-03+0.j,
       -7.76444118e-03+0.j,  7.76444118e-03+0.j, -7.76444118e-03+0.j, 7.76444118e-03+0.j,  8.47039180e-02+0.j, -9.01204279e-03+0.j,
       -9.01204279e-03+0.j,  9.01204279e-03+0.j, -9.01204279e-03+0.j, -6.57574490e-03+0.j,  6.57574490e-03+0.j, -6.57574490e-03+0.j,
        6.57574490e-03+0.j,  6.05056057e-02+0.j,  1.08894077e-02+0.j, 1.08894077e-02+0.j,  1.14339547e-01+0.j, -1.05401874e-02+0.j,
       -1.05401874e-02+0.j,  3.51167704e-02+0.j, -3.51167704e-02+0.j, -6.05056057e-02+0.j,  1.08894077e-02+0.j, -1.08894077e-02+0.j,
       -1.14091635e-01+0.j]
)

In [80]:
def compare_pauli_ops(op1: SparsePauliOp, op2: SparsePauliOp) -> bool:
    pairs_1 = list(zip([str(p) for p in op1.paulis], op1.coeffs))
    pairs_2 = list(zip([str(p) for p in op2.paulis], op2.coeffs))
    if len(pairs_1) != len(pairs_2):
        return False
    
    pairs_1.sort(key = lambda x: x[0])
    pairs_2.sort(key = lambda x: x[0])
    for pair_1, pair_2 in zip(pairs_1, pairs_2):
        if pair_1[0] != pair_2[0]:
            return False
        if np.abs(pair_1[1] - pair_2[1] > 1e-8):
            return False
        
    return True

In [81]:
print(compare_pauli_ops(H2_qub, H2_ham_0_75))
print(compare_pauli_ops(HeH_qub, HeH_ham_1))
print(compare_pauli_ops(LiH_qub, LiH_ham_1_5))

True
True
True
